In [ ]:
# default_exp reward

# Reward

> Reward function

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.chem import *
from mrl.templates import *
from mrl.agent import *

//anaconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
# export

class Reward():
    def __init__(self, template=None, reward_modules=[], trajectory_modules=[]):
        
        if template == None:
            template = Template([])
            
        self.template = template
        self.reward_modules = reward_modules
        self.trajectory_modules = trajectory_modules
        self.mean_reward = None
        
    def __call__(self, model_output):
        
        template_passes = np.array(np.array(self.template(model_output['sequences'])))
        template_rewards = np.array(self.template.eval_mols(model_output['sequences']))
        
        rewards = self.compute_rewards(model_output, template_passes)
        trajectory_rewards = self.compute_trajectory_reward(model_output, template_passes)
        
        rewards = template_rewards + rewards
        
        if self.mean_reward is None:
            self.mean_reward = rewards.mean()
        else:
            self.mean_reward = (1-reward_decay)*rewards.mean() + reward_decay*self.mean_reward
            
        rewards_scaled = rewards - self.mean_rewards
        
        model_output['rewards'] = rewards
        model_output['rewards_scaled'] = rewards_scaled
        model_output['trajectory_rewards'] = trajectory_rewards
        
        return model_output
    
    def compute_trajectory_reward(self, model_output, template_passes):
        
        all_rewards = []
        
        for rm in self.trajectory_modules:
            all_rewards.append(rm(model_output, template_passes))
            
        all_rewards = np.stack(all_rewards, -1)
        all_rewards = all_rewards.sum(-1)
        return all_rewards
    
    def compute_rewards(self, model_output, template_passes):
        
        all_rewards = []
        
        for rm in self.reward_modules:
            all_rewards.append(rm(model_output, template_passes))
            
        all_rewards = np.stack(all_rewards, -1)
        all_rewards = all_rewards.sum(-1)
        return all_rewards

In [ ]:
# export

def trajectory_wrapper(inputs, function):
    return np.array([function(i) for i in inputs])

In [ ]:
# export

class RewardModule():
    
    def __call__(self, model_output, template_passes=None):
        
        reward_inputs = self.prepare_reward_inputs(model_output, template_passes)
        reward_outputs = self.reward_function(reward_inputs)
        final_reward = self.aggregate_reward(reward_outputs, model_output, template_passes)
        return final_reward
        
    def aggregate_reward(self, reward_outputs, model_output, template_passes=None):
        pass
        
    def prepare_reward_inputs(self, model_output, template_passes=None):
        pass
    
    def reward_function(self, inputs):
        pass
    
class MolReward(RewardModule):
    def __init__(self, mol_function, trajectory=False):
        self.mol_function = mol_function
        self.trajectory = trajectory
        
    def aggregate_reward(self, reward_outputs, model_output, template_passes=None):
        
        if template_passes is not None:
            passed_idxs = np.array([i for i in range(len(template_passes)) if template_passes[i]])
            bs = len(template_passes)
        else:
            passed_idxs = np.arange(len(reward_outputs))
            bs = len(reward_outputs)
        
        if self.trajectory:
            outputs = np.zeros((bs, model_output['sl']))
            
            for i, idx in enumerate(passed_idxs):
                traj = reward_outputs[i]
                traj_len = len(traj)
                outputs[idx, :traj_len] = traj
                
        else:
            outputs = np.zeros((bs))
            outputs[passed_idxs] = reward_outputs
            
        return outputs
        
    def prepare_reward_inputs(self, model_output, template_passes=None):
        
        if self.trajectory:
            inputs = model_output['sequence_trajectories']
        else:
            inputs = model_output['sequences']
            
        output = np.zeros((len(inputs)))
        
        if template_passes is not None:
            inputs = [inputs[i] for i in range(len(inputs)) if template_passes[i]]
            
        return inputs
    
    def reward_function(self, inputs):
        if self.trajectory:
            func = partial(trajectory_wrapper, function=self.mol_function)
        else:
            func = self.mol_function
            
        return maybe_parallel(func, inputs)
    
class MLReward():
    def __init__(self, model, trajectory=False):
        self.model = model
        self.trajectory = trajectory
        
    def reward_function(self, inputs):
        if not type(inputs)==list:
            inputs = [inputs]
        return np.array(self.model(*inputs).detach().cpu())
    
    def prepare_reward_inputs(self, model_output, template_passes=None):
        raise NotImplementedError
        
    def aggregate_reward(self, reward_outputs, model_output, template_passes=None):
        if template_passes is not None:
            passed_idxs = np.array([i for i in range(len(template_passes)) if template_passes[i]])
            bs = len(template_passes)
        else:
            passed_idxs = np.arange(len(reward_outputs))
            bs = len(reward_outputs)
        
        if reward_outputs.ndim==2:
            output = np.zeros((bs, reward_outputs.shape[-1]))
            output[passed_idxs] = reward_output
            
        else:
            output = np.zeros((bs,))
            output[passed_idxs] = reward_outputs
            
        return output
    
class FPModelReward(MLReward):
    def __init__(self, model, fp_func, trajectory=False):
        super().__init__(model, trajectory)
        self.fp_func = fp_func
        
    def prepare_reward_inputs(self, model_output, template_passes=None):
        
        smiles = model_output['sequences']
        fps = np.stack(maybe_parallel(self.fp_func, smiles))
        fps = to_device(torch.from_numpy(fps).float())
        return fps
    
class SequenceModelReward(MLReward):

    def prepare_reward_inputs(self, model_output, template_passes=None):
        
        return model_output['x']

In [ ]:
def mf(smile):
    mol = to_mol(smile)
    if mol is None:
        output = 0.
    else:
        output = qed(mol)
        
    return output

In [ ]:
r = MolReward(mf, trajectory=True)

In [ ]:
mo = ModelOutput()

In [ ]:
mo['sequences'] = ['C', 'CCC', 'CCCC']
mo['sequence_trajectories'] = [['C'], ['C', 'CC', 'CCC'], ['C', 'CC', 'CCC', 'CCCC']]
mo['sl'] = 4
template_passes = np.array([True, False, True])

In [ ]:
r(mo, template_passes)

array([[0.35978494, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.35978494, 0.37278556, 0.38547066, 0.43102436]])

In [ ]:
r.prepare_reward_inputs(mo)

[['C'], ['C', 'CC', 'CCC'], ['C', 'CC', 'CCC', 'CCCC']]

In [ ]:
len(r.reward_function(r.prepare_reward_inputs(mo)))

3

In [ ]:
np.array([[5]]).ndim

2

In [ ]:
np.array([['C'], ['C', 'CC', 'CCC'], ['C', 'CC', 'CCC', 'CCCC']], dtype=object).ndim

1

parallel reward
    parallel process calculation on one sequence at a time
    
batch reward
    parallel featurize
    batch
    compute